# Novelty Algo

## Algorithm to Automate Novelty Scoring

### Import Packages

In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

import shared_functions as sf
from shared_functions import *

### Put Data from Excel Sheet into Dataframes

In [2]:
# individual df's for each sheet

# when on pc
data_test_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv")
data_test_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv")
data_test_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv")
data_test_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv")
data_test_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv")
data_test_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv")
data_test_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv")

# when on mac
# data_test_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_cup_semdis.csv.xlsx")
# data_test_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_key_semdis.csv.xlsx")
# data_test_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_rope_semdis.csv.xlsx")
# data_test_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_brick_semdis.csv.xlsx")
# data_test_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_chair_semdis.csv.xlsx")
# data_test_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_pencil_semdis.csv.xlsx")
# data_test_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/novelty/test/semdis/autdata_test_shoe_semdis.csv.xlsx")

In [3]:
# when on pc
data_official_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_cup_semdis.csv")
data_official_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_key_semdis.csv")
data_official_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_rope_semdis.csv")
data_official_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_brick_semdis.csv")
data_official_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_chair_semdis.csv")
data_official_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_pencil_semdis.csv")
data_official_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_shoe_semdis.csv")
data_official_box = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/novelty/official/semdis/autdata_official_box_semdis.csv")

# when on mac
# data_official_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_cup_semdis.csv.xlsx")
# data_official_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_key_semdis.csv.xlsx")
# data_official_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_rope_semdis.csv.xlsx")
# data_official_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_brick_semdis.csv.xlsx")
# data_official_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_chair_semdis.csv.xlsx")
# data_official_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_pencil_semdis.csv.xlsx")
# data_official_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_shoe_semdis.csv.xlsx")
# data_official_box = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/official/semdis/autdata_official_box_semdis.csv.xlsx")

### General Functions

In [4]:
stopwords = sf.stopwords_edited

## Semantic Spaces
### Word2Vec Models for Embeddings

In [5]:
# load pretrained model
word_model_twitter25 = api.load("glove-twitter-25")


In [6]:
# create dictionary of counts for each word in model
twitter25_dict = {}
for i in range(len(word_model_twitter25)):
    twitter25_dict[word_model_twitter25.index_to_key[i]] = word_model_twitter25.key_to_index[word_model_twitter25.index_to_key[i]]

In [7]:
# get the frequency of each word in dictionary
total_words = 0
for key in twitter25_dict:
    total_words = total_words + twitter25_dict[key]
    
for key in twitter25_dict:
    twitter25_dict[key] = twitter25_dict[key]/total_words

### ukwac subtitles Semantic Space

In [9]:
# when on pc
ukwac_subtitles = pd.read_csv('C:/Users/jhec8/Documents/Northwestern_SROP/cbow_6_ukwac_subtitle.txt', delimiter = " ", header = None)

# when on mac
# ukwac_subtitles_sem_space = pd.read_csv('/Users/johnhenrycruz/Desktop/Northwestern_SROP/cbow_6_ukwac_subtitle.txt', delimiter = " ", header = None, encoding='latin-1')

ukwac_subtitles = ukwac_subtitles.set_index(0)


In [10]:
ukwac_subtitles

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
biennials,0.018570,-0.115743,0.052710,-0.103825,-0.003274,-0.111418,-0.178500,-0.168650,-0.059751,-0.139651,...,0.121335,-0.021174,-0.113822,-0.110219,0.002543,-0.079831,0.103625,0.188465,0.117148,0.035641
fawn,0.071241,0.080455,0.216394,-0.282252,-0.172025,-0.186543,0.040009,0.044497,-0.045530,-0.079626,...,0.134902,-0.007096,-0.046929,0.022484,-0.042331,0.082441,0.077913,0.417162,-0.313959,-0.040423
gai,-0.183570,-0.039461,0.036953,-0.330616,-0.290676,0.095041,0.187567,0.019878,-0.115750,0.078831,...,0.042250,-0.009011,-0.311596,-0.131843,0.078453,0.048432,-0.005434,-0.001165,-0.072534,-0.151484
nunnery,-0.037334,0.180116,0.046011,-0.454054,-0.102627,-0.037614,-0.081269,-0.144102,0.018661,-0.312565,...,0.099364,-0.011798,-0.315447,0.160066,-0.179072,0.085825,-0.018394,0.055811,0.460388,0.107053
icici,0.010815,-0.050479,0.164478,-0.035702,-0.192204,0.003913,-0.063542,-0.095691,-0.131409,-0.090370,...,-0.040970,-0.021843,-0.071962,-0.120538,-0.018662,-0.263689,-0.119164,-0.013069,0.030275,-0.072135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
proteasome,0.065590,0.063595,-0.068297,-0.341265,-0.363577,-0.028856,0.121471,-0.163342,0.164599,-0.033824,...,0.092268,-0.019238,0.032392,0.024969,0.011309,0.061474,0.066890,-0.148575,-0.067886,0.027040
jawbone,-0.198237,-0.221238,0.034535,-0.125550,0.261507,-0.176464,0.154252,-0.001421,-0.012786,0.101746,...,0.173170,-0.018633,-0.150698,-0.278048,-0.083264,-0.138730,0.011749,0.292462,-0.002925,0.116922
expands,-0.204973,-0.104741,0.371615,-0.127140,-0.024127,-0.217444,-0.088636,0.147861,-0.060367,-0.200520,...,0.093103,0.040245,-0.231791,-0.101625,0.162721,-0.040362,0.097418,-0.007231,0.107961,-0.058140


### CBOW subtitles Semantic Space

In [11]:
# when on pc
cbow_subtitles = pd.read_csv('C:/Users/jhec8/Documents/Northwestern_SROP/cbow_subtitle.txt', delimiter = " ", header = None)

# when on mac
# ukwac_subtitles_sem_space = pd.read_csv('/Users/johnhenrycruz/Desktop/Northwestern_SROP/cbow_subtitle.txt', delimiter = " ", header = None, encoding='latin-1')

cbow_subtitles = cbow_subtitles.set_index(0)


In [12]:
cbow_subtitles

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
fawn,-0.092878,0.096702,0.078285,0.141875,-0.048455,0.022562,-0.116627,-0.133138,-0.006373,0.065410,...,-0.141192,0.033917,-0.091994,-0.198885,0.086601,-0.084499,0.128204,0.130868,0.034637,-0.011805
gai,-0.073987,-0.135876,0.071925,0.050427,-0.117428,0.119181,0.000579,0.058539,-0.164071,-0.039671,...,0.011994,0.046088,0.127052,0.037256,-0.099584,-0.093139,-0.116162,0.049216,-0.186436,0.006694
impotents,-0.002924,-0.016872,0.020181,0.015595,-0.016781,-0.014786,-0.007104,-0.016849,-0.026617,0.000086,...,0.045714,-0.037038,0.003953,-0.012306,0.007597,0.014559,-0.013585,-0.008614,0.016856,0.032169
sonja,0.046202,0.124720,0.111161,0.241513,-0.163221,0.119522,-0.011830,0.045682,0.213233,0.141021,...,-0.117941,0.133043,-0.038438,-0.089926,0.061131,-0.143988,-0.058695,0.110329,-0.069613,-0.142646
dionysian,0.032223,-0.007911,-0.008691,-0.015038,-0.037716,0.028007,-0.026402,0.023947,-0.036913,0.033985,...,-0.022568,-0.009473,0.001342,-0.056279,-0.001985,-0.037703,-0.027222,0.015461,-0.016171,0.015194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jawbone,-0.155908,-0.022339,0.029578,0.061183,-0.016605,-0.026134,0.004525,0.002993,-0.120240,0.077789,...,0.005585,-0.052877,0.068886,-0.118713,-0.161840,0.027309,-0.031582,0.072750,-0.000247,-0.041733
bunches,-0.032415,0.140536,0.071025,0.022989,0.031107,0.122717,-0.029894,0.092296,-0.071899,-0.097607,...,-0.010892,-0.053674,-0.071419,-0.081352,-0.032544,-0.096401,-0.006610,0.108215,0.034687,0.185569
zinka,-0.006705,0.002355,0.012616,0.021504,-0.002360,0.017963,-0.008720,-0.000567,-0.014686,-0.000533,...,-0.003436,0.006517,-0.006647,-0.009388,0.003580,-0.006105,0.006303,0.012804,0.005335,0.014729


### Banori Semantic Space

In [13]:
# when on pc
banori = pd.read_csv('C:/Users/jhec8/Documents/Northwestern_SROP/banori.txt', delimiter = " ", header = None)

# when on mac
# ukwac_subtitles_sem_space = pd.read_csv('/Users/johnhenrycruz/Desktop/Northwestern_SROP/banori.txt', delimiter = " ", header = None, encoding='latin-1')

banori = banori.set_index(0)

In [14]:
banori

,1,2,3,4,5,6,7,8,9,10,...,391,392,393,394,395,396,397,398,399,400
0,,,,,,,,,,,,,,,,,,,,,
the,-0.060292,0.067630,-0.036891,0.066684,0.024045,0.099091,0.009682,-0.099609,0.081138,-0.153635,...,0.011333,-0.072486,-0.116943,0.071367,0.002056,0.041920,0.017046,0.038706,0.033797,0.016488
",",0.026625,0.073101,-0.027073,-0.019504,0.041730,0.038811,0.094878,-0.031282,0.093890,-0.105021,...,-0.002796,0.029393,-0.069047,0.083530,-0.033128,-0.035646,0.027183,0.003230,0.000811,0.022505
.,-0.005893,0.093791,0.015333,0.046226,0.032791,0.110069,0.055551,-0.080625,0.150372,-0.121523,...,0.076293,-0.048817,-0.203812,0.012218,0.039033,-0.043925,0.087199,0.089214,0.073069,0.075056
of,-0.050371,0.031452,0.040910,0.033255,-0.009195,0.061086,0.085859,-0.122968,0.068290,-0.108840,...,-0.021496,-0.112018,-0.116068,0.071437,-0.041942,0.061624,0.030890,0.013635,-0.104569,0.005263
and,0.005456,0.063237,-0.075793,-0.000819,0.003407,0.053554,0.070145,-0.088482,0.129797,-0.094823,...,0.100012,-0.090594,-0.090813,0.028959,0.024793,-0.040878,-0.024795,0.083654,-0.010910,0.027067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
scientifica,-0.121726,0.094667,0.025527,0.001748,0.087804,-0.004830,-0.011430,-0.068062,0.016917,0.031084,...,0.082360,-0.025217,0.008511,-0.109005,-0.034504,0.027565,0.115459,0.000142,-0.022383,0.042615
schoolly,0.015603,0.100056,-0.042075,-0.032139,0.024201,0.051871,-0.069512,-0.144775,0.110104,-0.106074,...,0.014019,-0.040083,0.088833,-0.015218,-0.063183,0.021593,-0.101472,0.023395,0.022891,0.036009
schnapf,-0.057437,0.181676,-0.063820,0.017294,0.002916,0.019541,0.080525,-0.054740,-0.071306,-0.007485,...,0.079774,-0.068784,0.087408,-0.044859,-0.046738,-0.022576,0.023053,0.026027,0.010763,0.063920


### GloVe 6B Semantic Space

In [15]:
# when on pc
glove_6B = pd.read_csv('C:/Users/jhec8/Documents/Northwestern_SROP/glove_6B.txt', delimiter = " ", header = None)

# when on mac
# ukwac_subtitles_sem_space = pd.read_csv('/Users/johnhenrycruz/Desktop/Northwestern_SROP/glove_6B.txt', delimiter = " ", header = None, encoding='latin-1')

glove_6B = glove_6B.set_index(0)


In [16]:
glove_6B

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
the,0.046560,0.213180,-0.007436,-0.458540,-0.035639,0.236430,-0.288360,0.215210,-0.134860,-1.641300,...,-0.013064,-0.296860,-0.079913,0.195000,0.031549,0.285060,-0.087461,0.009061,-0.209890,0.053913
",",-0.255390,-0.257230,0.131690,-0.042688,0.218170,-0.022702,-0.178540,0.107560,0.058936,-1.385400,...,0.075968,-0.014359,-0.073794,0.221760,0.146520,0.566860,0.053307,-0.232900,-0.122260,0.354990
.,-0.125590,0.013630,0.103060,-0.101230,0.098128,0.136270,-0.107210,0.236970,0.328700,-1.678500,...,0.060148,-0.156190,-0.119490,0.234450,0.081367,0.246180,-0.152420,-0.342240,-0.022394,0.136840
of,-0.076947,-0.021211,0.212710,-0.722320,-0.139880,-0.122340,-0.175210,0.121370,-0.070866,-1.572100,...,-0.366730,-0.386030,0.302900,0.015747,0.340360,0.478410,0.068617,0.183510,-0.291830,-0.046533
to,-0.257560,-0.057132,-0.671900,-0.380820,-0.364210,-0.082155,-0.010955,-0.082047,0.460560,-1.847700,...,-0.012806,-0.597070,0.317340,-0.252670,0.543840,0.063007,-0.049795,-0.160430,0.046744,-0.070621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chanty,0.392700,-0.022505,0.304580,0.187990,0.141180,0.724030,-0.257810,-0.137290,-0.016521,0.595960,...,-0.182950,0.406630,-0.343630,-0.270400,-0.593680,0.016447,0.140740,0.463940,-0.369570,-0.287180
kronik,0.136790,-0.139090,-0.360890,0.079864,0.321490,0.263870,-0.109900,0.044420,0.083869,0.791330,...,0.036419,-0.036845,-0.348150,0.064732,-0.000577,-0.133790,0.428960,-0.023320,0.410210,-0.393080
rolonda,0.075713,-0.040502,0.183450,0.512300,-0.228560,0.839110,0.178780,-0.713010,0.326900,0.695350,...,-0.388530,0.545850,-0.035050,-0.184360,-0.197000,-0.350030,0.160650,0.218380,0.309670,0.437610


### TASA Semantic Space

In [17]:
# when on pc
TASA = pd.read_csv('C:/Users/jhec8/Documents/Northwestern_SROP/TASA.txt', delimiter = " ", header = None)

# when on mac
# ukwac_subtitles_sem_space = pd.read_csv('/Users/johnhenrycruz/Desktop/Northwestern_SROP/TASA.txt', delimiter = " ", header = None, encoding='latin-1')

TASA = TASA.set_index(0)


In [18]:
TASA

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
who,0.058924,-0.003555,0.053929,0.037707,-0.028096,-0.025736,0.008540,-0.041124,-0.000275,0.002969,...,0.002788,-0.005665,0.008141,0.035108,-0.005556,0.001101,0.022807,-0.001179,-0.004761,0.000947
were,0.076660,0.007832,0.102716,-0.046048,0.006808,-0.012557,0.012856,-0.043569,0.004773,0.027316,...,0.000811,0.003786,-0.011847,0.010002,0.027371,0.010266,-0.000217,0.005904,-0.005155,-0.017014
the,0.037283,-0.011585,0.003933,-0.006798,0.012415,0.000927,0.004378,-0.003837,0.006263,-0.002052,...,0.000049,0.000939,-0.001295,0.000544,0.000071,-0.000817,0.001123,-0.000980,-0.000354,0.000634
first,0.049274,-0.012220,0.023342,0.002829,0.008936,0.004214,0.041426,-0.014812,0.020341,0.018361,...,0.027615,0.016552,0.052426,-0.001099,-0.024695,-0.008158,-0.026864,-0.000865,-0.030933,-0.035347
americans,0.017895,-0.024743,0.068235,-0.003476,-0.022697,-0.002701,-0.013814,0.041591,0.008424,0.019061,...,0.009282,0.029840,-0.002187,-0.029926,-0.036506,0.008290,0.003097,0.028012,-0.002667,0.043664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
phosphagens,0.000049,-0.000144,-0.000252,-0.000134,0.000194,-0.000358,-0.000433,0.000109,0.000242,0.000393,...,-0.000659,-0.000874,-0.000879,-0.000478,-0.001883,0.000523,-0.001007,-0.000753,0.000633,-0.000265
phosphated,0.000025,-0.000072,-0.000126,-0.000067,0.000097,-0.000179,-0.000216,0.000054,0.000121,0.000197,...,-0.000330,-0.000437,-0.000439,-0.000239,-0.000941,0.000262,-0.000504,-0.000376,0.000317,-0.000132
adp,0.000025,-0.000072,-0.000126,-0.000067,0.000097,-0.000179,-0.000216,0.000054,0.000121,0.000197,...,-0.000330,-0.000437,-0.000439,-0.000239,-0.000941,0.000262,-0.000504,-0.000376,0.000317,-0.000132


## Novelty Algo 1
### Avg Distance using Word2Vec
### Lesser Average Similarity, More Novel
### More of a Proof of Concept Method

In [19]:
# method to get the novelty rating
# average of the similarities seen
def get_similarity_word2vec_avg(prompt, phrase_list, word_model):
    avg_sim = 0
    # find similarity of each word in phrase with the prompt
    for term in range(len(phrase_list)):
        avg_sim = avg_sim + word_model.similarity(w1 = prompt, w2 = phrase_list[term])
        
    # take the average
    if len(phrase_list) == 0:
        avg_sim = 0
    else:
        avg_sim = avg_sim/len(phrase_list)

    return (1 - avg_sim)

In [20]:
# method that returns the dataframe with novelty rating for each phrase
def get_novelty_word2vec_avg(df, prompt, stopwords_list, word_model, join_list):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    avg_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        avg_sim_list.append(get_similarity_word2vec_avg(prompt, response, word_model))

    # add novelty rating list to dataframe
    novel_rating_df['avg_sim'] = avg_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [21]:
data_test_brick_novelty_word2vec_avg = get_novelty_word2vec_avg(data_test_brick, "brick", stopwords, word_model_twitter25, False)
data_test_brick_novelty_word2vec_avg

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,avg_sim
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],0.465017
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],0.369515
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],0.334537
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",0.269556
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",0.257584
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",0.317669
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",0.283718
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",0.269556
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",0.257584
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",0.442764


## Novelty Algo 2
### Word2Vec + Smooth Inverse Frequency + Cosine Similarity

### Does work well because phrases are super short
### Don't use
### Fix if have time

In [22]:
# method to apply SIF to the vectors 
def get_sif_feature_vectors(prompt, response, word_model):
    # set the word count dictionary with frequencies
    word_counts = twitter25_dict
    # size of vectore in word embeddings
    embedding_size = 25 
    # set hyper parameter
    a = 0.001
    # list to store vectors
    phrase_set = []
    for phrase in [prompt, response]:
        # zero out the vector
        vs = np.zeros(embedding_size)
        phrase_length = len(phrase)
        for word in phrase:
            # smooth inverse frequency, SIF
            a_value = a / (a + word_counts[word]) 
            # vs += sif * word_vector
            vs = np.add(vs, np.multiply(a_value, word_model[word]))
        # weighted average
        if phrase_length == 0:
            vs[:] = 0
        else:
            vs = np.divide(vs, phrase_length) 
        phrase_set.append(vs)
    # return the SIF adjusted vectors
    return phrase_set

In [24]:
def get_similarity_word2vec_sif_cosinesim(prompt, response, word_model):
    # get the SIF adjusted vectors
    vectors = get_sif_feature_vectors(prompt, response, word_model)
    # return the cosine similarity
    return (get_cosine_distance(vectors[0], vectors[1]))

In [25]:
def get_novelty_word2vec_sif_cosinesim(df, prompt, stopwords_list, word_model, join_list):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    avg_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        avg_sim_list.append(get_similarity_word2vec_sif_cosinesim(prompt, response, word_model))

    # add novelty rating list to dataframe
    novel_rating_df['SIF + cosine sim'] = avg_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [53]:
data_test_brick_novelty_word2vec_sif_cosinesim = get_novelty_word2vec_sif_cosinesim(data_test_brick, "brick", stopwords, word_model_twitter25, False)
data_test_brick_novelty_word2vec_sif_cosinesim

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,SIF + cosine sim
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],0.563437
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],0.777387
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],0.515435
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",0.342759
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",0.398735
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",0.503658
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",0.283702
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",0.342759
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",0.398735
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",0.582402


## Novelty Algo 3
### ukwac_sem_space + element wise multiplication + cosine distance
### Greater Cos Distance, Greater Novelty
### Most similar to SemDis

In [27]:
# method to get the element wise multiplied vector
# multiply vectors in phrase
def get_ew_multiplied_vector(phrase_list, sem_space):
    vectors_list = []
    # add vectors to list
    # change to numpy array
    for term in phrase_list:
        vectors_list.append(np.array(sem_space.loc[term].values.tolist()))
    
    # get element wise multiplied vector
    element_wise_multiplied_vector = np.ones(len(sem_space.columns))

    for vector in vectors_list:
        element_wise_multiplied_vector = element_wise_multiplied_vector * vector

    return element_wise_multiplied_vector

In [28]:
# get cosine sim from prompt and ewm
def get_cosine_sim_ewm(prompt, response, sem_space):
    prompt_vector = np.array(sem_space.loc[prompt].values.tolist())
    ewm_vector = get_ew_multiplied_vector(response, sem_space)

    return (sf.get_cosine_distance(prompt_vector, ewm_vector))

In [29]:
# get df with results of the cosine distance from prompt using the elementwise multiplied vectors in the response
def get_novelty_ewm(df, prompt, stopwords_list, sem_space, join_list):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    # list to store cosine sims
    cosine_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list.append(get_cosine_sim_ewm(prompt, response, sem_space))

    # add novelty rating list to dataframe
    novel_rating_df['ewm_vector_cosine_dis'] = cosine_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [54]:
data_test_brick_novelty_ukwac_ewm_cosinedist = get_novelty_ewm(data_test_brick, 'brick', stopwords, ukwac_subtitles, False)
data_test_brick_novelty_ukwac_ewm_cosinedist

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,ewm_vector_cosine_dis
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],0.774175
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],0.850109
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],0.829263
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",0.938542
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",0.969920
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",0.995112
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",0.993950
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",0.938542
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",0.969920
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",1.054378


## Novelty Algo 4
### ukwac_sem_space + local minina + cosine distance

In [36]:
# get word in phrase that has the least distance from the prompt
def get_minvec(prompt, phrase_list, sem_space):
    distances_list = []
    # get prompt vector
    prompt_vector = np.array(sem_space.loc[prompt].values.tolist())
    
    # create list of cosine distances
    for term in phrase_list:
        distances_list.append(sf.get_cosine_distance(prompt_vector, np.array(sem_space.loc[term].values.tolist())))
        
    # return the max cosine distance
    return max(distances_list, default=0)

In [39]:
# get df with results of the cosine distance from prompt using the minima vector in the response
def get_novelty_minvec(df, prompt, stopwords_list, sem_space, join_list):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    # list to store cosine sims
    cosine_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list.append(get_minvec(prompt, response, sem_space))

    # add novelty rating list to dataframe
    novel_rating_df['minima_vector_cosine_dis'] = cosine_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [40]:
data_test_brick_novelty_ukwac_minimavec_cosinedist = get_novelty_minvec(data_test_brick, 'brick', stopwords_edited, ukwac_subtitles, False)
data_test_brick_novelty_ukwac_minimavec_cosinedist

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,minima_vector_cosine_dis
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],0.774175
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],0.850109
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],0.829263
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",0.680430
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",0.809089
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",0.880264
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",0.880264
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",0.680430
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",0.809089
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",0.918658


## Novelty Algo 5
### ukwac_sem_space + element wise multiplication + cosine distance + clustering
### average responses cosine distance in the same cluster
### idea is that phrases with same alternate task will group
### variation in phrase in the same cluster will  be averaged out

In [42]:
# averages the distance of the phrases in each cluster
# gives each phrase in cluster the average distance 
def get_clustered_novelty_score(novel_rating_df, column, num_clusters, display):
    # get cluster df
    clusters_df = sf.get_counts_vector(num_clusters, novel_rating_df['response_processed_phrase'].tolist(), display)
    # get cleaned phrases and their current novelty rating
    novelty_scores = dict(zip(novel_rating_df.response_processed_phrase, novel_rating_df[column]))

    # create dictionary out of cluster df
    clusters = dict(zip(clusters_df.category, clusters_df.responses))
        
    # initialize empty dictionary to store the score for a category
    clusters_scores = dict.fromkeys(clusters)
    
    # get the average cosine distance for a cluster
    for key in clusters:
        score = 0
        for phrase in clusters[key]:
            score = score + novelty_scores[phrase]
        score = score/len(clusters[key])
        clusters_scores[key] = score
        
    # create dictionary to store a phrase and its new novelty score 
    # new score is the average of the responses in one cluster
    phrase_scores_dict = {}
    for key in clusters:
        for phrase in clusters[key]:
            phrase_scores_dict[phrase] = clusters_scores[key]
            
    # make a list that matches the one in the current dataframe
    # return list to be added to dataframe
    df_phrases_scores_list = [] 
    for phrase in novel_rating_df['response_processed_phrase'].tolist():
        df_phrases_scores_list.append(phrase_scores_dict[phrase])
    
    # uncomment to show clusters df
#     display(clusters_df)
            
    return list(df_phrases_scores_list)

In [43]:
# get df with results of the cosine distance from prompt using the elementwise multiplied vectors in the response
def get_novelty_ewm_cluster(df, prompt, stopwords_list, sem_space, join_list, num_clusters, display):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    novel_rating_df['response_processed_phrase'] = [' '.join(x) for x in cleaned_responses]
    # list to store cosine sims
    cosine_sim_list = []
    
    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list.append(get_cosine_sim_ewm(prompt, response, sem_space))

    # add novelty rating list to dataframe
    novel_rating_df['ewm_vector_cosine_dis'] = cosine_sim_list
    
    novel_rating_df['ewm_vector_cosine_dis_clus_avg'] = get_clustered_novelty_score(novel_rating_df, 'ewm_vector_cosine_dis', num_clusters, display)
    
    # new column with novelty rating
    return novel_rating_df

In [55]:
data_test_brick_novelty_ukwac_ewm_cosinedist_cluster = get_novelty_ewm_cluster(data_test_brick, 'brick', stopwords, ukwac_subtitles, False, 22, False)
data_test_brick_novelty_ukwac_ewm_cosinedist_cluster

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,response_processed_phrase,ewm_vector_cosine_dis,ewm_vector_cosine_dis_clus_avg
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],foundation,0.774175,0.893464
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],weapon,0.850109,0.850109
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],bowl,0.829263,0.893464
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",build house,0.938542,0.938542
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",break window,0.969920,0.964829
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",line fireplace,0.995112,0.907823
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",line road,0.993950,0.907823
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",build house,0.938542,0.938542
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",break window,0.969920,0.964829
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",weight workout,1.054378,1.015204


## Novelty Algo 6
### ukwac_sem_space + local minima + cosine distance + clustering
### average responses cosine distance in the same cluster
### idea is that phrases with same alternate task will group
### variation in phrase in the same cluster will  be averaged out
### differs from algo 5, does local minima not ewm

In [49]:
# get df with results of the cosine distance from prompt using the elementwise multiplied vectors in the response
def get_novelty_minvec_cluster(df, prompt, stopwords_list, sem_space, join_list, num_clusters, display):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    novel_rating_df['response_processed_phrase'] = [' '.join(x) for x in cleaned_responses]
    # list to store cosine sims
    cosine_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list.append(get_minima_vector_cos_distance(prompt, response, sem_space))

    # add novelty rating list to dataframe
    novel_rating_df['minima_vector_cosine_dis'] = cosine_sim_list
        
    novel_rating_df['minima_vector_cosine_dis_clus_avg'] = get_clustered_novelty_score(novel_rating_df, 'minima_vector_cosine_dis', num_clusters, display)
    
    # new column with novelty rating
    return novel_rating_df

In [56]:
data_test_brick_novelty_ukwac_minimavec_cosinedist_cluster = get_novelty_minvec_cluster(data_test_brick, 'brick', stopwords, ukwac_subtitles, False, 22, False)
data_test_brick_novelty_ukwac_minimavec_cosinedist_cluster

,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,response_processed_phrase,minima_vector_cosine_dis,minima_vector_cosine_dis_clus_avg
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],foundation,0.774175,0.804059
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],weapon,0.850109,0.850109
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],bowl,0.829263,0.829263
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",build house,0.680430,0.680430
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",break window,0.809089,0.809089
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",line fireplace,0.880264,0.804059
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",line road,0.880264,0.804059
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",build house,0.680430,0.680430
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",break window,0.809089,0.809089
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",weight workout,0.918658,0.918658


## Novelty Algo 7
### uses the cosine + minima + clustering methods
### uses the same clusterr for all algos
### two different scoring systems, average or minimum

In [63]:
# averages the distance of the phrases in each cluster
# gives each phrase in cluster the average distance
# generalized to avg or min, both ewm and minima
def get_clustered_novelty_score_generalized(clusters_df, novel_rating_df, average, column):
    # get cleaned phrases and their current novelty rating
    novelty_scores = dict(zip(novel_rating_df.response_processed_phrase, novel_rating_df[column]))

    # create dictionary out of cluster df
    clusters = dict(zip(clusters_df.category, clusters_df.responses))
        
    # initialize empty dictionary to store the score for a category
    clusters_scores = dict.fromkeys(clusters)
    
    # get the average or min cosine distance for a cluster
    if average:
        for key in clusters:
            score = 0
            for phrase in clusters[key]:
                score = score + novelty_scores[phrase]
            score = score/len(clusters[key])
            clusters_scores[key] = score
    else:
        for key in clusters:
            scores_list = []
            for phrase in clusters[key]:
                scores_list.append(novelty_scores[phrase])
            clusters_scores[key] = min(scores_list)
        
    # create dictionary to store a phrase and its new novelty score 
    # new score is the average of the responses in one cluster
    phrase_scores_dict = {}
    for key in clusters:
        for phrase in clusters[key]:
            phrase_scores_dict[phrase] = clusters_scores[key]
            
    # make a list that matches the one in the current dataframe
    # return list to be added to dataframe
    df_phrases_scores_list = [] 
    for phrase in novel_rating_df['response_processed_phrase'].tolist():
        df_phrases_scores_list.append(phrase_scores_dict[phrase])
    
    # uncomment to show clusters df
#     display(clusters_df)
            
    return list(df_phrases_scores_list)

In [64]:
# get df with results of the cosine distance from prompt using the elementwise multiplied vectors in the response
def get_novelty_combined(df, prompt, stopwords_list, sem_space, join_list, num_clusters, display):
    # clean the responses
    novel_rating_df = sf.get_cleaned_responses_df(df, stopwords_list, join_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    novel_rating_df['response_processed_phrase'] = [' '.join(x) for x in cleaned_responses]
    # list to store cosine sims for ewm
    cosine_sim_list_ewm = []
    # list to store cosine sims for minima
    cosine_sim_list_minima = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list_ewm.append(get_cosine_sim_ewm(prompt, response, sem_space))
        cosine_sim_list_minima.append(get_minima_vector_cos_distance(prompt, response, sem_space))
        
     # get clusters for the dataset
    # idea is to use the same clusters for each analysis
    clusters_df = get_counts_vector(num_clusters, novel_rating_df['response_processed_phrase'].tolist(), display)

    # add novelty rating list to dataframe for ewm
    novel_rating_df['ewm_vector_cosine_dis'] = cosine_sim_list_ewm
    
    # add the columns for the novelty scores
    novel_rating_df['ewm_vector_cosine_dis_clus_avg'] = get_clustered_novelty_score_generalized(clusters_df, novel_rating_df, True, 'ewm_vector_cosine_dis')

    # add novelty rating list to dataframe for minima
    novel_rating_df['minima_vector_cosine_dis'] = cosine_sim_list_minima
    novel_rating_df['minima_vector_cosine_dis_clus_avg'] = get_clustered_novelty_score_generalized(clusters_df, novel_rating_df, True, 'minima_vector_cosine_dis')
    novel_rating_df['minima_vector_cosine_dis_clus_min'] = get_clustered_novelty_score_generalized(clusters_df, novel_rating_df, False, 'minima_vector_cosine_dis')

    
    # new column with novelty rating
    return novel_rating_df

In [65]:
data_test_brick_novelty_combined = get_novelty_combined(data_test_brick, 'brick', stopwords, ukwac_subtitles, False, 22, True)
data_test_brick_novelty_combined

,category,responses
0,0,"[build, build, build, build, build]"
1,1,"[weapon, weapon, weapon, weapon, weapon]"
2,2,"[paperweight, paperweight, paperweight, paperw..."
3,6,[paint]
4,7,[break]
5,8,[house]
6,9,"[throw demolish, throw, throw]"
7,10,"[line road, road]"
8,11,"[foundation, bowl, karate, fire pit, sidewalk,..."
9,12,[thrown weapon]


,id,response,response_nofill,item,item_nofill,SemDis_cbowukwacsubtitle_nf_m,response_processed,response_processed_phrase,ewm_vector_cosine_dis,ewm_vector_cosine_dis_clus_avg,minima_vector_cosine_dis,minima_vector_cosine_dis_clus_avg,minima_vector_cosine_dis_clus_min
0,1476,as a foundation,foundation,Brick,brick,0.77417,[foundation],foundation,0.774175,0.880566,0.774175,0.817691,0.670092
1,1476,as a weapon,weapon,Brick,brick,0.85011,[weapon],weapon,0.850109,0.850109,0.850109,0.850109,0.850109
2,1476,as a bowl,bowl,Brick,brick,0.82926,[bowl],bowl,0.829263,0.880566,0.829263,0.817691,0.670092
3,1718,build a house,build house,Brick,brick,0.93854,"[build, house]",build house,0.938542,0.938542,0.680430,0.680430,0.680430
4,1718,break a window,break window,Brick,brick,0.96992,"[break, window]",break window,0.969920,0.969920,0.809089,0.809089,0.809089
5,1718,line a fireplace,line fireplace,Brick,brick,0.99511,"[line, fireplace]",line fireplace,0.995112,0.995112,0.880264,0.880264,0.880264
6,1718,line a road,line road,Brick,brick,0.99395,"[line, road]",line road,0.993950,0.864178,0.880264,0.807335,0.734406
7,1691,build house,build house,Brick,brick,0.93854,"[build, house]",build house,0.938542,0.938542,0.680430,0.680430,0.680430
8,1691,break window,break window,Brick,brick,0.96992,"[break, window]",break window,0.969920,0.969920,0.809089,0.809089,0.809089
9,1691,weight for workout,weight workout,Brick,brick,1.05438,"[weight, workout]",weight workout,1.054378,0.982614,0.918658,0.918658,0.918658


### Write Results into Excel Sheet

In [53]:
# parallel lists for the test datasets
prompts_list = ['cup', 'key', 'rope', 'brick', 'chair', 'pencil', 'shoe']
data_list = [data_test_cup, data_test_key, data_test_rope, data_test_brick, data_test_chair, data_test_pencil, data_test_shoe]

In [54]:
# write results df of each dataset for specific methods and semantic spaces
def write_results_excel(method, stopwords, model):
    # change this when writing new sheet
    writer = pd.ExcelWriter('novelty_algo_6_results.xlsx', engine='xlsxwriter')

    for i in range(len(prompts_list)):
#         df = method(data_list[i], prompts_list[i], stopwords, model, True, False)
        df = method(data_list[i], prompts_list[i], stopwords, True, False)
        df.to_excel(writer, sheet_name = prompts_list[i], index = False)
    writer.save()

In [55]:
# write_results_excel(get_novelty_word2vec_avg, stopwords_spacy, word_model_twitter25)

In [56]:
# write_results_excel(get_novelty_word2vec_sif_cosinesim, stopwords_spacy, word_model_twitter25)

In [57]:
# write_results_excel(get_novelty_ukwac_ewm_cosinedist, stopwords_spacy, word_model_twitter25)

In [58]:
# write_results_excel(get_novelty_ukwac_minimavec_cosinedist, stopwords_spacy, word_model_twitter25)

In [59]:
# write_results_excel(get_novelty_ukwac_ewm_cosinedist_cluster, stopwords_spacy, word_model_twitter25)

In [60]:
# write_results_excel(get_novelty_ukwac_minimavec_cosinedist_cluster, stopwords_spacy, word_model_twitter25)

### Write Results into CSV

In [61]:
# list to iterate through
# note that they are in the same orderr
prompts_list = ['box', 'brick', 'chair', 'cup', 'key', 'pencil', 'rope', 'shoe']
data_list = [data_official_box, data_official_brick, data_official_chair, data_official_cup, data_official_key, data_official_pencil, data_official_rope, data_official_shoe]
sem_space_list = [ukwac_subtitles, cbow_subtitles, banori, TASA, glove_6B]
sem_space_list_str = ['ukwac_subtitles', 'cbow_subtitles', 'banori', 'TASA', 'glove_6B']

In [62]:
def write_novelty_results_csv():
    for y in range(len(sem_space_list)):
        for i in range(len(prompts_list)):
#             ewm_df = get_novelty_ukwac_ewm_cosinedist_cluster(data_list[i], prompts_list[i], stopwords_edited, sem_space_list[y], True, False)
#             min_df = get_novelty_ukwac_minimavec_cosinedist_cluster(data_list[i], prompts_list[i], stopwords_edited, sem_space_list[y], True, False)
#             df = pd.concat([ewm_df, min_df[min_df.columns[-2:]]], axis=1)
            # this line uses the same cluster for all methods
            df = get_novelty_combined(data_list[i], prompts_list[i], stopwords_edited, sem_space_list[y], True, False)
            df.rename(columns={'rating':'novelty_1'}, inplace=True)
            df.rename(columns={'creativity':'novelty_2'}, inplace=True)
            df['novelty_m'] = df[['novelty_1', 'novelty_2']].mean(axis=1)
            # reorder cols the way I want them 
            df = df[['id',
                 'response',
                 'response_nofill',
                 'response_processed',
                 'response_processed_phrase',
                 'item',
                 'item_nofill',
                 'SemDis_factor',
                 'SemDis_cbowukwacsubtitle_nf_m',
                 'SemDis_cbowsubtitle_nf_m',
                 'SemDis_cbowBNCwikiukwac_nf_m',
                 'SemDis_TASA_nf_m',
                 'SemDis_glove_nf_m',
                 'SemDis_MEAN',
                 'ewm_vector_cosine_dis',
                 'ewm_vector_cosine_dis_clus_avg',
                 'minima_vector_cosine_dis',
                 'minima_vector_cosine_dis_clus_avg',
                 'minima_vector_cosine_dis_clus_min',
                 'novelty_1',
                 'novelty_2',
                 'novelty_m']]
            df.to_csv(prompts_list[i] + "_" + sem_space_list_str[y] + "_" + "results" + ".csv", index=False) 

In [63]:
# write_novelty_results_csv()

Algo Design Brainstorming:
* Word2vec, average similarity between words in two responses
* Problems:
    * phrases vs words
    * compare between just two responses or the whole list of responses
* Algo Idea
    * find similarity between prompt and average of word in response
    * the average is the value that represents novelty
* Things to Look Into:
    * doc2Vec - look into sentence vector
        * follow similar logic to above but no need to average for multiple words
    * Word2Vec + SIF + Cosine Similarity
    * Word2Vec + WMD 
* Course of Actions for Missing Words/Misspellings?
    * hand remove?
* What to do when whole phrase stop words
    * make 0

To Do List
- [x] write preprocessing methods
- [x] write out initial algo
- [x] implement first algo idea
- [x] set up work environment on macbook
- [x] look into Word2Vec + SIF + Cosine Similarity
- [x] set up excel sheet download with different sheets
- [x] use ukwac semantic space for cosine distance
- [x] set up element wise multiplied vectors for cosine distance algo
- [x] set up phrase minima vector for cosine distance algo 
- [x] Compare the ewm and phrase minima algos with SemDis
- [x] Get a sense of the effect of the compositions
- [x] 1 - to get the distance
- [x] Remove the word could 
    - had to lowercase first and then remove stop words
- [x] write clustering method
    - count vectorizer and kmeans
- [x] update clustering method to use cosine distance not euclidean 
    - changed from scikit learn to cosine distance from NLTK 
- [x] write algo to average novelty scores of responses in the same category
    - do for both ewm and minima
- [x] use the elbow method to figure out how many clusters to use
- [x] download and upload all the semantic spaces from SemDis
- [x] update methods to change semantic spaces
- [x] copy semantic spaces to mac
- [x] set up scripts to pass in official data for pc and mac
- [x] add .DS_Store to gitignore
- [x] get SemDis results for all 6 different semantic spaces with the official data
- [ ] compare performance on the 6 different semantic spaces
    - correlation tests for coefficient and significance
- [x] update stop words list to include "use" and "thing"
- [ ] figure out how many times to run kmeans
    - cross validation
    - then averaging the results of all the iterations
- [ ] figure out way to automate the number of clusters used in each run
 
Don't look into unless you have time
- [ ] look into doc2vec 
- [ ] look into Word2Vec + WMD 
- [ ] look at https://github.com/PrincetonML/SIF for better SIF 
